## Daten Laden

In [1]:
file_path = "BIG_2025-03-11__22_08_29-normalized-cleaned-supplemented.csv"
print(file_path)

BIG_2025-03-11__22_08_29-normalized-cleaned-supplemented.csv


In [2]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

,watt-median,L1-dcache-loads,L1-dcache-loads-root,L1-dcache-loads-square,L1-dcache-stores,L1-dcache-stores-root,L1-dcache-stores-square,L1-icache-load-misses,L1-icache-load-misses-root,L1-icache-load-misses-square,...,unc_m_dram_page_empty_rd-square,unc_m_vc0_requests_rd,unc_m_vc0_requests_rd-root,unc_m_vc0_requests_rd-square,unc_mc0_rdcas_count_freerun,unc_mc0_rdcas_count_freerun-root,unc_mc0_rdcas_count_freerun-square,unc_m_dram_page_hit_rd,unc_m_dram_page_hit_rd-root,unc_m_dram_page_hit_rd-square
0,2.215,4.271935e+04,206.686590,1.824943e+09,2.304721e+04,151.813067,5.311738e+08,7.330692e+03,85.619461,5.373905e+07,...,4.024427e+10,6.741319e+05,821.055344,4.544538e+11,3.373781e+05,580.842561,1.138240e+11,4.607492e+05,678.785093,2.122898e+11
1,14.245,2.398101e+09,48970.407054,5.750887e+18,9.903185e+08,31469.327009,9.807308e+17,1.292005e+07,3594.447148,1.669277e+14,...,5.287053e+12,6.650065e+06,2578.771931,4.422336e+13,3.442425e+06,1855.377269,1.185029e+13,4.339139e+06,2083.060050,1.882813e+13
2,11.340,5.822350e+08,24129.546767,3.389976e+17,2.023007e+08,14223.244788,4.092557e+16,9.467402e+04,307.691429,8.963169e+09,...,5.010760e+15,8.035012e+07,8963.822965,6.456142e+15,4.017730e+07,6338.556601,1.614215e+15,7.356387e+06,2712.265947,5.411642e+13
3,11.970,1.005903e+09,31715.975961,1.011841e+18,4.542079e+08,21312.154951,2.063049e+17,1.020875e+07,3195.112898,1.042185e+14,...,9.764023e+08,8.962852e+04,299.380232,8.033272e+09,4.526592e+04,212.757901,2.049004e+09,5.760171e+04,240.003565,3.317957e+09
4,11.800,1.465998e+09,38288.354584,2.149150e+18,1.463533e+09,38256.144836,2.141928e+18,3.281931e+05,572.881397,1.077107e+11,...,1.054228e+09,9.310908e+04,305.137798,8.669300e+09,4.692420e+04,216.619955,2.201881e+09,5.969025e+04,244.315889,3.562926e+09


## Korrelationsmatrix

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

/opt/conda/lib/python3.12/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/opt/conda/lib/python3.12/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


## 1. Versuch (Regression)

In [4]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Zielvariable: Annahme, dass die erste Spalte die zu schätzende Zahl enthält
y = df.iloc[:, 0]

# Feature, das immer genutzt werden soll: 'instructions'
X_instructions = df[['instructions']]

# Potenzielle zusätzliche Features: Alle Spalten außer der Zielvariable und 'instructions'
potential_features = df.drop(columns=[df.columns[0], 'instructions'])

# Automatische Auswahl der 4 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=4)
X_selected = selector.fit_transform(potential_features, y)

# Ausgabe der Namen der ausgewählten Features
selected_feature_names = potential_features.columns[selector.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Falls gewünscht: Umwandeln in DataFrame, um die Spaltennamen beizubehalten
X_selected_df = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Kombination der 'instructions'-Spalte mit den ausgewählten Features
X_combined = pd.concat([X_instructions, X_selected_df], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Training eines Regressionsmodells (z. B. Lineare Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


Ausgewählte Features: ['cpu-cycles-root', 'ref-cycles-square', 'msr/mperf/-square', 'msr/tsc/-square']
Mean Squared Error: 34.35630635480962


## 2. Versuch (Regression)

### Modell trainieren

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Zielvariable: erste Spalte (angenommen, es handelt sich um numerische Werte)
y = df.iloc[:, 0]

# Fixes Feature: 'instructions'
X_instructions = df[['instructions']]

# Kandidaten für die automatische Feature-Auswahl:
# Entfernen der Zielvariable und der Spalte 'instructions'
X_candidates = df.drop(columns=[df.columns[0], 'instructions'])

# Basisregressor für die Feature-Selektion (hier: Lineare Regression)
model = LinearRegression()

# Sequential Feature Selector (forward selection) wählt 4 zusätzliche Features aus
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select=4,
    direction='forward',
    scoring='neg_mean_squared_error',
    cv=5  # 5-fache Kreuzvalidierung
)

# Durchführung der Feature-Selektion auf den Kandidatenfeatures
sfs.fit(X_candidates, y)

# Ermitteln der Namen der ausgewählten Features
selected_feature_names = X_candidates.columns[sfs.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Zusammenführen des fixen Features 'instructions' mit den ausgewählten Features
X_selected = pd.concat([X_instructions, X_candidates[selected_feature_names]], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Modelltraining auf den kombinierten Features
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Ausgabe der Regressionsparameter
print("\nRegression Parameter:")
print("Achsenabschnitt (Intercept):", model.intercept_)

# Ausgabe der Koeffizienten mit den zugehörigen Feature-Namen
print("Koeffizienten:")
for feature, coef in zip(X_selected.columns, model.coef_):
    print(f"{feature}: {coef}")

Ausgewählte Features: ['dTLB-loads-root', 'cpu-cycles-root', 'instructions-root', 'slots']
Mean Squared Error: 0.27446118135696196

Regression Parameter:
Achsenabschnitt (Intercept): 3.680585979456649
Koeffizienten:
instructions: 1.0636746743835474e-10
dTLB-loads-root: 4.0735310953370876e-05
cpu-cycles-root: 8.970233253645297e-05
instructions-root: -6.586042551045461e-06
slots: 7.462134724831081e-11


### Modell testen

In [6]:
def do_regression(instructions, dTLB_loads, cache_misses, cstate_core_c6_residency, unc_m_clockticks):
  return -417.5947581136508 + instructions * 4.65216312028892e-11 + dTLB_loads * 7.883112920541098e-10 + cache_misses * -1.9612014211040394e-07 + cstate_core_c6_residency * 4.18166881063541e-09 + unc_m_clockticks * 1.3126082256029556e-07

# sleep 10 (expected 2.125)
print("sleep 10 (expected 2.125)")
print(do_regression(168777.863665457, 41575.009771487, 1361.36887244182, 27904585074.6458, 177728131.709414))

print("")

# sum_up_benchmark (expected 14.505)
print("sum_up_benchmark (expected 14.505)")
print(do_regression(11664606808.4586, 2175275696.69778, 6728517.43140718, 28062502860.874, 2390791162.75018))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 16.39)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 16.39)")
print(do_regression(15248513772.5695, 2013098946.16777, 56550.6387563183, 27893525904.2613, 2389444709.36587))

sleep 10 (expected 2.125)
-277.5785106389854

sum_up_benchmark (expected 14.505)
14.5084025767747

stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 16.39)
14.972450838656641


## 3. Versuch (Schwierig auf Colab...)

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomSelector(BaseEstimator, TransformerMixin):
    """
    Transformer, der die Spalte 'instructions' immer mit einbezieht
    und zusätzlich eine Liste von weiteren Features auswählt, die als Parameter übergeben wird.
    """
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Es werden immer 'instructions' plus die angegebenen features zurückgegeben
        return X[['instructions'] + list(self.features)]

def optimal_feature_selection_regression_cv(df, test_size=0.2, random_state=42):
    """
    Verwendet GridSearchCV, um die optimale Feature-Kombination zu finden.
    'instructions' wird stets genutzt und zusätzlich werden 4 weitere Features
    aus den Kandidaten (alle außer der Zielvariable und 'instructions') ausgewählt.

    Rückgabe:
      dict: Mit den besten zusätzlichen Features, dem CV-Score,
            Test-MSE, Achsenabschnitt und Koeffizienten.
    """
    # Zielvariable: erste Spalte
    y = df.iloc[:, 0]
    # Prädiktoren: alle Spalten außer der Zielvariable
    X = df.drop(columns=[df.columns[0]])

    # Kandidaten für zusätzliche Features: alle außer 'instructions'
    candidates = list(X.drop(columns=['instructions']).columns)

    # Pipeline: CustomSelector + LinearRegression
    pipeline = Pipeline([
        ('selector', CustomSelector(features=[])),
        ('regressor', LinearRegression())
    ])

    # Erzeuge den Parametergrid: alle Kombinationen von 4 Features aus den Kandidaten
    param_grid = {
        'selector__features': [list(comb) for comb in combinations(candidates, 4)]
    }

    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X, y)

    best_features = grid.best_params_['selector__features']
    best_score = grid.best_score_

    print("Beste zusätzliche Features:", best_features)
    print("Bester CV Score (neg_mean_squared_error):", best_score)

    # Aufteilen in Trainings- und Testdaten
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Bestes Modell trainieren
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Finales Test-MSE:", mse)

    # Regressionsparameter ausgeben
    regressor = best_model.named_steps['regressor']
    print("\nRegression Parameter:")
    print("Achsenabschnitt (Intercept):", regressor.intercept_)

    coef_dict = {}
    # Die verwendeten Spalten: 'instructions' plus die best_features
    features_used = ['instructions'] + best_features
    print("Koeffizienten:")
    for feature, coef in zip(features_used, regressor.coef_):
        print(f"{feature}: {coef}")
        coef_dict[feature] = coef

    return {
        "best_features": best_features,
        "cv_score": best_score,
        "test_mse": mse,
        "intercept": regressor.intercept_,
        "coefficients": coef_dict
    }

# Beispiel zur Nutzung:
# df = pd.read_csv('data.csv')
# result = optimal_feature_selection_regression_cv(df)
# print(result)

optimal_feature_selection_regression_cv(df)